# 任务 2：使用 SageMaker Experiments

在本实验中，您将使用 Amazon SageMaker Experiments 设置一个试验。您将使用 XGBoost 训练机器学习 (ML) 模型，执行超参数优化以测试多个超参数设置和生成更准确的模型，并评估模型的性能。

## 任务 2.1：环境设置

在开始训练模型之前，请安装所有必要依赖项。


要了解有关 SageMaker Experiments 的功能的详情，请参阅 [使用 Amazon SageMaker Experiments 管理机器学习](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html#experiments-features)。

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.experiments.run import Run, load_run
#from sagemaker.utils import unique_name_from_base  #could be used instead of the date-time append approach, to create a unique Experiment name.
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlasms'

接下来，导入数据集。

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

将数据集拆分为训练数据集 (70%)、验证数据集 (20%) 和测试数据集 (10%)。在训练期间将使用训练数据集和验证数据集。在部署后进行模型评估时将使用测试数据集。

要使用 SageMaker 进行训练，您需要将数据集转换为 libSVM 或 CSV 格式。本实验使用 CSV 格式进行训练。

有关 XGBoost 算法的信息，请参阅 [XGBoost 算法](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)。
有关 XGBoost 算法的输入/输出界面的更多信息，请参阅 [XGBoost 算法的输入/输出界面](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost)。


In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

您已经创建了两个数据集文件，名称为 *train_data.csv* 和 *validation_data.csv*。
将这些数据集文件上传到 Amazon Simple Storage Service (Amazon S3)。

In [ ]:
#upload-dataset
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## 任务 2.2：创建试验并运行初始训练作业

使用 SageMaker Experiments，通过各种训练组件来整理、跟踪、比较和评估 ML 模型训练试验。有关 SageMaker Experiments 的更多信息，请参阅 [SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html)。在 SageMaker Experiments 中，这些组件包括数据集、算法、超参数和指标。

在此任务中，您将完成以下操作：
- 在 Amazon SageMaker Studio 中创建和跟踪试验。
- 创建一个运行来跟踪输入、参数和指标。

首先，为试验创建名称并提供描述。

In [ ]:
#create unique experiment name
create_date = strftime("%m%d%H%M")

lab_6_experiment_name = "lab-6-{}".format(create_date)
description = "Using SageMaker Experiments with the Adult dataset."




然后，针对运行名称和标签定义可选值。

In [ ]:
# create initial run_name
run_name = "lab-6-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-6', 'Value': 'lab-6-run'}]

print(f"Experiment name - {lab_6_experiment_name},  run name - {run_name}")

## 任务 2.3：使用 XGBoost 算法训练和优化模型

试验已设置好，可供训练使用。训练完成后，您可以在 SageMaker Studio 中分析结果。在此任务中，您将： 

- 训练 XGBoost 模型。
- 在 SageMaker Studio 中分析试验。
- 使用超参数优化模型。
- 在 SageMaker Studio 中分析优化结果。

### 任务 2.3.1：训练 XGBoost 模型

现在，您将使用 XGBoost 算法和您创建的试验来训练模型。

您将设置的超参数如下所示：
- **eta**：更新中使用的步长收缩，用于防止过度拟合。在每个提升步骤之后，您可以直接获得新特征的权重。
- **gamma**：在树的叶节点上进行进一步分区所需的最小损失减少。算法越大，越保守。
eta 参数实际上缩减了特征权重，使提升过程更加保守。
- **max_depth**：树的最大深度。增大此值会使模型更加复杂，并可能导致过度拟合。
- **min_child_weight**：子树中所需的实例权重 (hessian) 的最小总和。如果树分区步骤导致某个叶节点的实例权重总和小于 min_child_weight 值，则构建过程将放弃进一步分区。在线性回归模型中，这对应于每个节点所需的最小实例数。算法越大，越保守。
- **num_round**：用于提升的轮数（树）。增加树可以提高模型准确性，但会增加过度拟合的风险。
- **objective**：指定学习任务和相应的学习目标。
- **subsample**：训练实例的子样本比率。将其设置为 0.5 意味着 XGBoost 随机抽取一半的数据实例来建立树。这将防止过度拟合。
- **verbosity**：打印消息的详细程度。有效值为 0（静默）、1（警告）、2（信息）和 3（调试）。

运行训练大约需要 3–4 分钟。

有关 XGBoost 超参数的更多信息，请参阅 [超参数](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)。

In [ ]:
from sagemaker import image_uris
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')

# initialize hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8
verbosity=0

hyperparameters = {
        "max_depth":max_depth,
        "eta":eta,
        "gamma":gamma,
        "min_child_weight":min_child_weight,
        "subsample":subsample,
        "verbosity":verbosity,
        "objective":objective,
        "num_round":num_round
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    EnableSageMakerMetricsTimeSeries=True,
    hyperparameters=hyperparameters,
    tags = run_tags
)


#Run the training job link to Experiment.
with Run(
    experiment_name=lab_6_experiment_name,
    run_name=run_name,
    tags=run_tags,
    sagemaker_session=sagemaker_session,
) as run:

    run.log_parameters({
                        "eta": eta, 
                        "gamma": gamma, 
                        "max_depth": max_depth,
                        "min_child_weight": min_child_weight,
                        "num_round": num_round,
                        "objective": objective,
                        "subsample": subsample,
                        "verbosity": verbosity
                       })
    
#    you may also specify metrics to log
#    run.log_metric(name="", value=x)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    ) 

### 任务 2.3.2：评估优化前的模型性能

在 SageMaker Studio 中，您可以创建图表来评估您的训练作业。例如，在运行 lab-6 试验后，您能够以图表格式查看 validation:logloss_max 值。

在本实验中，您可以直接在笔记本中内联绘制其他指标。

In [ ]:
#visualize-training-results-table
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
plt.show()

### 任务 2.3.3：使用超参数优化模型

您已使用 SageMaker Experiments 成功执行了模型训练。在训练时，您还可以将 SageMaker 配置为使用超参数来显著影响训练过的模型的性能。SageMaker Studio 包含用于模型训练的各种常见超参数优化选项。虽然测试众多参数的效率会因所使用的数据集而异，但创建最佳模型也可能需要花费大量时间和精力。

SageMaker 自动模型优化功能可以自动选择超参数来优化训练。有关自动模型优化的更多信息，请参阅 [自动模型优化](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)。要使用该功能，您需要为选择用于优化的每个超参数指定一个范围或一个可能值的列表。SageMaker 自动模型优化功能将使用各种超参数设置自动运行多个训练作业。它随后将根据指定的目标指标评估每个作业的结果，并根据之前的结果选择要在将来尝试的超参数设置。对于每个优化作业，您将指定训练作业的最大数量，达到该数量后，优化作业就完成了。

您将需要设置的超参数范围如下所示：
- **alpha**：针对权重的 L1 正则项。增大此值会使模型更加保守。
- **eta**：更新中使用的步长收缩，用于防止过度拟合。在每个提升步骤之后，您可以直接获得新特征的权重。eta 参数实际上缩减了特征权重，使提升过程更加保守。
- **max_depth**：树的最大深度。增大此值会使模型更加复杂，并可能导致过度拟合。
- **min_child_weight**：子树中所需的实例权重 (hessian) 的最小总和。如果树分区步骤导致某个叶节点的实例权重总和小于 min_child_weight 值，则构建过程将放弃进一步分区。在线性回归模型中，这对应于每个节点所需的最小实例数。算法越大，越保守。
- **num_round**：用于提升的轮数（树）。增加树可以提高模型准确性，但会增加过度拟合的风险。

完成优化大约需要 5 分钟。

有关 XGBoost 超参数的更多信息，请参阅 [超参数范围](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)。

In [ ]:
#tune-model
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# Setup the hyperparameter ranges
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 2),
    'eta': ContinuousParameter(0, 1),
    'max_depth': IntegerParameter(1, 10),
    'min_child_weight': ContinuousParameter(1, 10),
    'num_round': IntegerParameter(100, 1000)
}
# Define the target metric and the objective type (max/min)
objective_metric_name = 'validation:auc'
objective_type='Maximize'
# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type='Auto',
)

with load_run(sagemaker_session=sagemaker_session, experiment_name=lab_6_experiment_name, run_name=run_name) as run:
# Tune the model
    tuner.fit(
        inputs = data_inputs,
        job_name = lab_6_experiment_name,
    )
    run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
 
plt.show()
    

### 任务 2.3.4：评估优化后的模型性能

在 SageMaker Studio 中，您还可以创建图表来评估您的优化作业。例如，在运行 lab-6-trial 训练作业后，您能够以图表格式查看目标值 **validation:auc_max**。

![SageMaker Studio 中的 validation:error_max 图表的图片。](Task_2_3_4.png)

在本实验中，查看来自最佳优化作业的结果，并使用笔记本中的图表对其进行可视化。

In [ ]:
#get_experiment_analytics 
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    sagemaker_session=Session(sess, sm),
)

run_component_analytics.dataframe()

In [ ]:
#visualize-tuning-results-auc-max
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    run_component_analytics.dataframe()["validation:auc - Max"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
else:
    run_component_analytics.dataframe()["validation:auc - Last"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
    
plt.show()

In [ ]:
#visualize-tuning-results-auc-max-scatter
N = 12
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Max"];
else:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Last"];
y = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["num_round"]

plt.scatter(x, y, alpha=0.5)
plt.title("auc_max by num_round")
plt.xlabel("validation:auc - Max")
plt.ylabel("num_round");
plt.show()

最后，您可以根据自己的目标指标打印最佳优化作业。

In [ ]:
#print-best
tuner.best_training_job()

### 任务 2.3.5：通过 SageMaker Studio 使用内置功能绘制试验指标

之前提到的方法使用内联笔记本单元格来创建试验指标图表。另一种选择是使用 SageMaker Studio 内的功能来绘制一些试验指标。现在，试验已经至少运行一次，可以在 SageMaker Studio 中创建一个新条形图。

下一项任务将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_6.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_6.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览构件时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完构件后，通过选择 **lab_6.ipynb** 选项卡返回至笔记本。

1.选择 **SageMaker Home**（SageMaker 主页）图标。
1.选择 **Experiments**（试验）。

此时 SageMaker Studio 将显示 **Experiments**（试验）选项卡。

1.选择以 *lab-6-* 开头的试验。

此时 SageMaker Studio 将显示该试验中包含的 **Runs**（运行）的列表。

1.选择与超参数优化作业关联的所有可用运行旁边的 **Name**（名称）列中的选项。
1.清除 
1.选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Analyze</span>（分析）。

此时 SageMaker Studio 将显示 **Run Analyze Chart**（运行分析图表）选项卡。

1.在选项卡下半部分的图表部分中，选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">+ Add Chart</span>（+ 添加图表）。
1.选择 **Bar**（条形图）。

此时 SageMaker Studio 将显示 **Add Chart**（添加图表）窗口。

1.对于 **Y-axis**（Y 轴），选择 **min_child_weight**。
1.选择 <span style="background-color:#73cdf9; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-radius:2px; border-width:3px; margin-right:5px; white-space:nowrap">Create</span>（创建）。

显示试验中每个 *run* 的 *min_child_weight* 的条形图现已保存到图表部分。

1.重复此流程，为 **train:auc** 指标创建一个新条形图。
1.重复此流程，为 **validation:auc** 指标创建一个新条形图。

### 总结

恭喜！ 您已使用 SageMaker Experiments 训练和优化模型。在下一个实验中，您将使用 SageMaker Debugger 来深入了解训练模型时的潜在问题。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。